## Code for analyzing the Today-Asahi dataset
#### Data Management (Spring/Summer 2018) at OSIPP, Osaka U

### Python version

### Preamble

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
from linearmodels.panel import PanelOLS
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
os.chdir("..") # set the parent directory as a working directory

In [17]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 200

### Import data

In [ ]:
ta_panel = pd.read_table('build_input/todai-asahi/output/syuuin_2009_2014_py.csv',sep=',') 
print(len(ta_panel))
#ta_panel = ta_panel.set_index(['uid','ELECYEAR']) # set index
ta_panel['const'] = 1 # add constant (which will be used for regressions)
ta_panel = ta_panel[ta_panel['PREFEC'] != 66] # remove PR
print(len(ta_panel))

### Check contents

In [ ]:
print(ta_panel.head(5)) # top 5 rows

In [ ]:
print(ta_panel.describe()) # summary

In [ ]:
print(ta_panel.isna().any()) #  missing values
#print(ta_panel[ta_panel['PREFEC'].isna() == True].describe())

### Make a summary table

In [34]:
ta_panel.agg(['mean','std','min','max','count']).T.to_latex("analysis_output/sum_stat.tex")
#ta_panel.describe().loc[['mean','std','min','max','count']].T.to_latex("analysis_output/sum_stat.tex")

#### - Group summary
- Useful method: `groupby()`.

In [ ]:
yn = ta_panel['yn_fiscalpol'].dropna() # remove missing values in yn_ficalpol
yn = yn[yn != 99] # remove "no answer"
print(len(yn), len(ta_panel))
result = yn.groupby(ta_panel['SEX']).describe() # summarize yn_fiscalpol by sex
print(result)

m = yn[(ta_panel['SEX'] == 1)]
f = yn[(ta_panel['SEX'] == 2)]
print(stats.ttest_ind(m, f)) # statistical difference between yn_fiscalpol for males and females(t-stat, assuming Normality and homogeneity of variance)

### Make figures

In [36]:
plt.style.use('seaborn') # set a plot style

#### - Histograms

In [ ]:
yn = ta_panel['yn_fiscalpol'].dropna()  
yn = yn[yn != 99]

plt.hist(yn, bins=5, align='mid', range=(0.5,5.5), alpha=0.3, color='k')
plt.title('Title')
plt.show()

In [ ]:
yn = ta_panel['yn_fiscalpol'].dropna() 
yn = yn[yn != 99]
m = yn[(ta_panel['SEX'] == 1)]
f = yn[(ta_panel['SEX'] == 2)]

# make a plot with two histograms
plt.subplot(1,2,1, title='Male',facecolor='white') # left figure
plt.hist(m, bins=5, align='mid', range=(0.5,5.5), alpha=0.3, color='b')

plt.subplot(1,2,2, title='Female',facecolor='white').set_ylim([0, 1050]) # right figure
plt.hist(f, bins=5, align='mid', range=(0.5,5.5), alpha=0.3, color='r')

plt.suptitle("Fiscal policy")
#plt.show()

plt.savefig("analysis_output/yn_fiscalpol_py.png") # save file

In [ ]:
# alternative code
yn = ta_panel['yn_fiscalpol'].dropna() 
yn = yn[yn != 99]
m = yn[(ta_panel['SEX'] == 1)]
f = yn[(ta_panel['SEX'] == 2)]

# make a plot with two histograms
fig = plt.figure()
left = fig.add_subplot(1,2,1, title='Male',facecolor='white') # left figure
right = fig.add_subplot(1,2,2, title='Female', ylim=(0,1050),facecolor='white') # right figure

left.hist(m, bins=5, align='mid', range=(0.5,5.5), alpha=0.3, color='b')
right.hist(f, bins=5, align='mid', range=(0.5,5.5), alpha=0.3, color='r')

plt.suptitle('Fiscal policy')
plt.show()

In [ ]:
# alternative code using seaborn's FacetGrid
ta_panel2 = ta_panel[:]
ta_panel2['yn_fiscalpol'] = ta_panel2['yn_fiscalpol'].dropna() 
ta_panel2 = ta_panel2[ta_panel2['yn_fiscalpol'] != 99]
ta_panel2['SEX'] = ta_panel2['SEX'].map({1: "Male", 2: "Female"})

g = sns.FacetGrid(ta_panel2, col='SEX', hue='SEX', hue_kws={'color': ['blue', 'red']})
g.map(plt.hist, 'yn_fiscalpol', bins=5, align='mid', range=(0.5,5.5), alpha=0.3)
g.fig.suptitle("Fiscal policy")
g.set_titles('{col_name}')

In [ ]:
# combine two figures in a single plot
yn = ta_panel['yn_fiscalpol'].dropna() 
yn = yn[yn != 99]
m = yn[(ta_panel['SEX'] == 1)]
f = yn[(ta_panel['SEX'] == 2)]

plt.hist(m, bins=5, align='mid', range=(0.5,5.5), alpha=0.3, color='b') 
plt.hist(f, bins=5, align='mid', range=(0.5,5.5), alpha=0.3, color='r')

plt.title('')
plt.show()

#### - Kernel density plots

In [ ]:
np.random.seed(123456789) # give a seed
rdm = pd.Series(np.random.normal(0,1,size=1000)) # get random values
rdm.hist(bins=100, alpha=0.3, color='r', normed=True) # plot a histogram
rdm.plot.kde(style='k--') # plot kernel density

#### - Scatter plots

In [ ]:
ta = ta_panel[(ta_panel['fav_ozawa'] != 999)] # remove "no answer"

ta.plot.scatter(y='AGE', x='fav_ozawa')
plt.show()

#### - Line plots

In [ ]:
np.random.seed(123456789) # give a seed
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(np.random.randn(1000).cumsum(), 'b', label='one') 
ax.plot(np.random.randn(1000).cumsum(), 'r', label='two')
ax.plot(np.random.randn(1000).cumsum(), 'y', label='three')
ax.legend(loc='best') # plot a legend

### Make regression tables

In [ ]:
# subset
ta1 = ta_panel[ta_panel['ab_asiaus'] != 99]

# define covariates
X1 = ['const', 'AGE']
X2 = ['const', 'AGE', 'PREFEC']

# OLS regressions 
reg1 = sm.OLS(ta1['ab_asiaus'], ta1[X1], missing='drop').fit() 
reg2 = sm.OLS(ta1['ab_asiaus'], ta1[X2], missing='drop').fit()

# summary of results
results_table = summary_col(results=[reg1,reg2],
                            float_format='%0.2f',
                            stars = True,
                            model_names=['Model 1',
                                         'Model 2'],
                            info_dict={'R-squared' : lambda x: "{:.3f}".format(x.rsquared),
                                       'Number of obs.' : lambda x: "{0:d}".format(int(x.nobs))},
                            regressor_order=['AGE',
                                             'PREFEC',
                                             'const'])

results_table.add_title('Table X - OLS Regressions')

print(results_table)

- You can use the formula interface for `statsmodels`.
    - Use `smf` instead of `sm`.

In [ ]:
# subset
ta1 = ta_panel[ta_panel['ab_asiaus'] != 99]

# OLS regressions
reg1 = smf.ols('ab_asiaus ~ AGE', data=ta1, missing='drop').fit()
reg2 = smf.ols('ab_asiaus ~ AGE + PREFEC', data=ta1, missing='drop').fit()

# summary of results
results_table = summary_col(results=[reg1,reg2],
                            float_format='%0.2f',
                            stars = True,
                            model_names=['Model 1',
                                         'Model 2'],
                            info_dict={'R-squared' : lambda x: "{:.3f}".format(x.rsquared),
                                       'Number of obs.' : lambda x: "{0:d}".format(int(x.nobs))},
                            regressor_order=['AGE',
                                             'PREFEC'])

results_table.add_title('Table X - OLS Regressions')

print(results_table)

# save the output table
file = open("analysis_output/ols_results.tex", "w")
file.write(results_table.as_latex())
file.close()

#### - Include Dummies

In [ ]:
# take a subset
ta1 = ta_panel[(ta_panel['yn_nkorea'] != 99) & (ta_panel['yn_preemp'] != 99)]

# OLS regressions with fixed effects (individuals and election years)
reg1 = smf.ols('yn_preemp ~ yn_nkorea', data=ta1, missing='drop').fit()
reg2 = smf.ols('yn_preemp ~ yn_nkorea + C(uid) + C(ELECYEAR)', data=ta1, missing='drop').fit()

# summary of results
results_table = summary_col(results=[reg1,reg2],
                            float_format='%0.3f',
                            stars = True,
                            model_names=['Model 1',
                                         'Model 2'],
                            info_dict={'R-squared' : lambda x: "{:.3f}".format(x.rsquared),
                                       'Number of obs.' : lambda x: "{0:d}".format(int(x.nobs))},
                            regressor_order=['yn_nkorea'],
                            drop_omitted=True) # Try True

results_table.add_title('Table X - OLS Regressions')

print(results_table)

- Alternatively, use `PanelOLS` in `linearmodels` (not recommended).

In [ ]:
ta1 = ta_panel[(ta_panel['yn_nkorea'] != 99) & (ta_panel['yn_preemp'] != 99)]

ta1 = ta1.set_index(['uid','ELECYEAR']) # set index

mod = PanelOLS.from_formula('yn_preemp ~ yn_nkorea + EntityEffects + TimeEffects', ta1)
#dep = ta1.yn_preemp
#exog = sm.add_constant(ta1[['yn_nkorea']])
#mod = PanelOLS(dep, exog, entity_effects=True, time_effects=True)

res = mod.fit(cov_type='unadjusted')
#res = mod.fit(cov_type='clustered', cluster_entity=True) # cluster standard errors at the entity level (individual level)
print(res)

- Cluster standard errors.

In [ ]:
# take a subset
ta1 = ta_panel[(ta_panel['yn_nkorea'] != 99) & (ta_panel['yn_preemp'] != 99)]
ta1 = ta1.dropna(subset=['yn_nkorea','yn_preemp']) # you have to drop NaN before computing clustered standard errors

# OLS regressions with clustered standard errors (at the individual level)
reg1 = smf.ols('yn_preemp ~ yn_nkorea', data=ta1).fit()
reg2 = smf.ols('yn_preemp ~ yn_nkorea', data=ta1).fit(cov_type='cluster', cov_kwds={'groups':ta1['uid']}) # clustered at the individual level

# summary of results
results_table = summary_col(results=[reg1,reg2],
                            float_format='%0.3f',
                            stars = True,
                            model_names=['Model 1',
                                         'Model 2'],
                            info_dict={'R-squared' : lambda x: "{:.3f}".format(x.rsquared),
                                       'Number of obs.' : lambda x: "{0:d}".format(int(x.nobs))},
                            regressor_order=['yn_nkorea'],
                            drop_omitted=True) 

results_table.add_title('Table X - OLS Regressions')

print(results_table)